In [ ]:
!pip -q install -U langchain langchain-google-genai pydantic python-dotenv

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving .env to .env


In [ ]:
!ls -la

total 24
drwxr-xr-x 1 root root 4096 Aug 25 11:26 .
drwxr-xr-x 1 root root 4096 Aug 25 10:19 ..
drwxr-xr-x 4 root root 4096 Aug 21 13:40 .config
-rw-r--r-- 1 root root   54 Aug 25 11:26 .env
drwxr-xr-x 3 root root 4096 Aug 25 11:10 mini_research_brief
drwxr-xr-x 1 root root 4096 Aug 21 13:41 sample_data


In [ ]:
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())

key = os.getenv("GOOGLE_API_KEY")
print("API Key loaded?", bool(key))
print("Key prefix:", (key[:6] + "…") if key else None)

API Key loaded? True
Key prefix: AIzaSy…


In [ ]:
import sys
from typing import List
from pydantic import BaseModel, ValidationError, field_validator
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

class ResearchBrief(BaseModel):
    title: str
    problem_statement: str
    key_questions: List[str]
    method_brief: List[str]
    deliverables: List[str]

    @field_validator("problem_statement")
    @classmethod
    def max_two_sentences(cls, v: str) -> str:
        count = sum(v.count(ch) for ch in ".!?")
        if count > 2:
            raise ValueError("problem_statement must be <= 2 sentences")
        return v.strip()

    @field_validator("key_questions")
    @classmethod
    def q_len(cls, v: List[str]) -> List[str]:
        if not (1 <= len(v) <= 3):
            raise ValueError("key_questions must have 1–3 items")
        return [s.strip() for s in v]

    @field_validator("method_brief")
    @classmethod
    def m_len(cls, v: List[str]) -> List[str]:
        if not (2 <= len(v) <= 4):
            raise ValueError("method_brief must have 2–4 items")
        return [s.strip() for s in v]

    @field_validator("deliverables")
    @classmethod
    def d_len(cls, v: List[str]) -> List[str]:
        if not (2 <= len(v) <= 3):
            raise ValueError("deliverables must have 2–3 items")
        return [s.strip() for s in v]


In [ ]:
def require_api_key():
    if not os.getenv("GOOGLE_API_KEY"):
        raise RuntimeError(
            "Missing GOOGLE_API_KEY. Set it using:\n"
            "os.environ['GOOGLE_API_KEY'] = 'your_key_here'"
        )

def build_chain():
    parser = PydanticOutputParser(pydantic_object=ResearchBrief)

    system_rules = (
        "You write ultra-concise, practical research briefs.\n"
        "Return ONLY JSON that matches this schema:\n{format_instructions}\n"
        "Rules:\n"
        "- Keep 'problem_statement' to <= 2 sentences.\n"
        "- 'key_questions' has 1–3 items.\n"
        "- 'method_brief' has 2–4 items.\n"
        "- 'deliverables' has 2–3 items.\n"
        "No markdown, no extra keys, no commentary."
    )

    prompt = (
        ChatPromptTemplate
        .from_messages([
            ("system", system_rules),
            ("human", "Topic: {topic}")
        ])
        .partial(format_instructions=parser.get_format_instructions())
    )

    model = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0,
        max_retries=2,
    )

    return prompt | model | parser

def to_markdown(brief: ResearchBrief) -> str:
    lines = []
    lines.append(f"# {brief.title}")
    lines.append(f"**Problem:** {brief.problem_statement}")
    lines.append("**Key Questions:**")
    lines.extend(f"- {q}" for q in brief.key_questions)
    lines.append("**Method (brief):**")
    lines.extend(f"- {m}" for m in brief.method_brief)
    lines.append("**Deliverables:**")
    lines.extend(f"- {d}" for d in brief.deliverables)
    return "\n".join(lines)


In [ ]:
require_api_key()
topic = input("Enter a research topic: ").strip()

if not topic:
    print("ERROR: Topic cannot be empty.", file=sys.stderr)
else:
    try:
        chain = build_chain()
        brief: ResearchBrief = chain.invoke({"topic": topic})

        print("\n JSON Output:")
        print(brief.model_dump_json(indent=2, ensure_ascii=False))

    except ValidationError as ve:
        print("ERROR: Model returned invalid JSON.", file=sys.stderr)
        print(ve, file=sys.stderr)
    except Exception as e:
        print(f"ERROR: {e}", file=sys.stderr)


Enter a research topic: impact of cat on human

 JSON Output:


ERROR: BaseModel.model_dump_json() got an unexpected keyword argument 'ensure_ascii'


In [ ]:
import json

result = chain.invoke({"topic": topic})

# JSON Output
json_output = json.dumps(result.model_dump(), indent=2, ensure_ascii=False)
print("JSON Output:")
print(json_output)

# Markdown Output
md_output = to_markdown(result)

print("\nMarkdown Output:")
print(md_output)

JSON Output:
{
  "title": "Impact of Cat Ownership on Human Well-being",
  "problem_statement": "Limited research exists on the comprehensive effects of cat ownership on human health.  This study will investigate the multifaceted impact of cat companionship on various aspects of human well-being.",
  "key_questions": [
    "How does cat ownership affect human mental health?",
    "What is the correlation between cat ownership and cardiovascular health?",
    "Does cat ownership influence social interaction and loneliness?"
  ],
  "method_brief": [
    "Conduct a literature review of existing studies on human-animal interaction.",
    "Survey cat owners to gather data on their experiences and health outcomes.",
    "Analyze survey data using statistical methods to identify correlations.",
    "Compare findings with existing research on other pet ownership models."
  ],
  "deliverables": [
    "A comprehensive report summarizing the findings.",
    "A presentation of key findings for a r